<a href="https://colab.research.google.com/github/LillyBal/PLS_SEM_Capstone/blob/main/PLSCleaningDataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning Data for PLS-SE Analysis


###Import data 
Import nummric csv file from Qualtrics & libraries pandas and numpy 


In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/User+acceptability_May+21,+2022_08.58.csv')

###Cleaning

1. Drop irrelevant columns
2. Drop answers of testing phase (14 respondents)
3. Drop answers that not consent


In [ ]:
data = data.drop(data.loc[:, 'StartDate':'Duration (in seconds)'].columns, axis = 1)
data = data.drop(data.loc[:, 'RecordedDate':'UserLanguage'].columns, axis = 1)

In [ ]:
data = data.drop(data.loc[:15].index)

In [ ]:
indexNames = data[  (data['Finished'] == 0) ].index
data.drop(indexNames , inplace=True)

###Create new Data frame
create new data frame and delete all demographical columns. Convert data points into numeric data type.

In [ ]:
df = pd.DataFrame(data)
df = df.drop(df.loc[:, 'Intro':'Blockchain_Knowledge_11_TEXT'].columns, axis = 1)
df = df.drop(df.loc[:, 'Q14':'Q19 Occupation_6_TEXT'].columns, axis = 1)
df = df.drop('Q19 Mail', axis = 1)

df = df.apply(pd.to_numeric)

Add demographic variables.

In [ ]:
df['Age'] = data['Q15_Age']
df['Gender'] = data['Q16 Gender']
df['Education'] = data['Q17 Education']
df['Occupation'] = data['Q19 Occupation']
df['Income'] = data['Q20 income']
df['Income_Descriptives'] = data['Q20 income']
df['Nationality'] = data['Q18 Nationality']
df['Blockchain_Knowledge'] = data['Blockchain_Knowledge']

### Missing values
Delete all rows with more than 3 missing values.

In [ ]:
df = df[df.isnull().sum(axis=1) > 3]

Create a dataframe containing nationality and blockchain_knowledge since they have no nummeric data (e.g. Germany and Dutch 3,2) 

In [ ]:
df2 = pd.DataFrame([df.Nationality, df.Blockchain_Knowledge]).transpose()
df2 = df2.reset_index(drop=True)
df2['Nationality'] = df2['Nationality'].fillna(0)
df = df.drop(df.loc[:, 'Nationality':'Blockchain_Knowledge'].columns, axis = 1)

Compute missing values with KNNImputer.

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(missing_values=np.nan, n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df),columns = df.columns)

Add Nationality and Blockchain_Knowledge to the data frame with the computed missing values.

In [ ]:
df = df.join(df2, how='left')
df.rename(columns = {'Q12 Social Facil_1':'Q12 Social_1', 'Q12 Social Facil_2':'Q12 Social_2', 'Q12 Social Facil_3':'Q12 Facilitating_1', 'Q12 Social Facil_4':'Q12 Facilitating_2', 'Q12 Social Facil_5':'Q12 Facilitating_3'}, inplace = True)


#Download

Download new files for visualisation and analysis.

In [ ]:
df.to_excel('data.xlsx')
df.to_csv('data.csv')